## 사용할 모듈 미리 로딩

In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib as mpl
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import time
import platform
import numpy as np
import random
import os
import shutil
from PIL import Image
import collections
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable


from sqlalchemy.sql import text as Text
from sqlalchemy import create_engine
from sqlalchemy import engine

## 데이터베이스(선택) 및 유저 에이전트

In [3]:

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

sqlinfo = engine.URL.create(
    drivername='mysql+pymysql',
    username = 'kdt5',
    password = 'q1@q1@q1@',
    database = 'webcrawling',
    query = {'charset':'utf8mb4'},
    host = "kdt5.mysql.database.azure.com",
)

conn = create_engine(sqlinfo,connect_args={'ssl':{"fake_flag_to_enable_tls":True}})
# cur 사용시 Text 로 쿼리문을 묶어 사용
# cur = conn.connect()
# result = cur.execute(Text('select * from editors'))
# for row in result.fetchall():
#     print(row)

## 폰트 설정

In [8]:
mpl.rc('font', family='Malgun Gothic')

## sql 판다스 연동하기

In [16]:
# 해당 구문을 활용하여 sql 에서 읽어보세요
# pd.read_sql('select * from editors', conn)

In [18]:
# 해당 구문을 활용하여 sql로 전송해주세요
# sqldf = pd.read_sql('select * from editors', conn)
# sqldf.to_sql('editors_copied'+str(random.randint(100,1000)), conn, if_exists='replace', index=False)

69

## requests 활용하기

아래처럼 session 으로 만든 다음에 사용하시면  
보다 빠르게 웹페이지 크롤링 가능

In [5]:
from urllib.parse import quote
import json
def replacecomma(x):
    return x.str.replace(',','')

corp_data=pd.read_csv('saraminDF_it_companies_final.csv', encoding='utf-8')
corp_list=corp_data['IT기업2'].to_list()

final_corp_list = []
final_corp_jsonTXT_list = []
for corp in corp_list:
    # corp = corp.strip('주식회사 ')
    # corp = corp.strip(' (주) ')
    
    # print(corp)
    parsed = quote(corp)
    url_BASE = 'https://www.catch.co.kr'
    
    href = '/Comp/CompSummary/I95823'
    test_link = f'https://www.catch.co.kr/Search/SearchList?Keyword={parsed}'
    
    
    with requests.session() as session:
        result = session.get(test_link)
        soup = BeautifulSoup(result.content,'lxml')
        detaillist = soup.select_one('ul.list_corp_round.atFull')
        try:
            sublink = detaillist.select('a')[0].attrs['href']
            final_corp_list.append(detaillist.select('a')[0].text.strip())
            print(sublink)
        except Exception as e:
            print('검색결과 없음')
            continue
        urljson = 'https://www.catch.co.kr/api/v1.0/comp/compSummary/getFinancial/{}'.format(sublink.split('/')[-1])
        result = session.get(urljson)
        jsonTxt = result.text
        if jsonTxt != 'null':
            final_corp_jsonTXT_list.append(jsonTxt)
        # else:
        #     final_corp_jsonTXT_list.append(None)
        # 
        # with open('test.json', 'w', encoding='utf-8') as f:
        #     f.write(result.text)
        # 
        

/Comp/CompSummary/G26372
/Comp/CompSummary/A99AB
/Comp/CompSummary/H07854
/Comp/CompSummary/HS6109
/Comp/CompSummary/ABCD2
/Comp/CompSummary/A73E2
/Comp/CompSummary/C48351
/Comp/CompSummary/038618
/Comp/CompSummary/B0013
/Comp/CompSummary/556556
/Comp/CompSummary/497730
/Comp/CompSummary/C20187
검색결과 없음
/Comp/CompSummary/487787
/Comp/CompSummary/JB2183
/Comp/CompSummary/024518
/Comp/CompSummary/495146
/Comp/CompSummary/A88AE
/Comp/CompSummary/JB6124
/Comp/CompSummary/465936
/Comp/CompSummary/C46482
검색결과 없음
/Comp/CompSummary/123950
/Comp/CompSummary/G29102
/Comp/CompSummary/J77049
검색결과 없음
/Comp/CompSummary/133430
/Comp/CompSummary/HO3684
/Comp/CompSummary/M15136
/Comp/CompSummary/IS8879
/Comp/CompSummary/L25425
/Comp/CompSummary/023670
/Comp/CompSummary/388799
/Comp/CompSummary/I12933
/Comp/CompSummary/620475
/Comp/CompSummary/HG2578
/Comp/CompSummary/JW6339
/Comp/CompSummary/F15568
/Comp/CompSummary/JM7677
/Comp/CompSummary/M29394
검색결과 없음
검색결과 없음
/Comp/CompSummary/JH0600
/Comp/CompSumma

In [75]:
# import pickle
# 
# with open('final_corp_jsonTXT_list.pickle', 'wb') as fw:
#     pickle.dump(final_corp_jsonTXT_list, fw)
#     
# with open('final_corp_jsonTXT_list.pickle', 'rb')as fr:
#     final_corp_jsonTXT_list = pickle.load(fr)
import gc
gc.collect()

empty_df = pd.DataFrame(columns=[])

'''
계정 목록 (더 있는데 뭐가 더 있는지 찾기 어려운 상태
compID','YMD', 'FnType',"유동자산", "비유동자산", "자산총계", "유동부채", "비유동부채", "부채총계", "자본금", "자본총계", "매출액", "매출총이익", "영업이익", "영업외수익", "영업외비용", "법인세차감전이익", "법인세비용", "총당기순이익", "총자산증가율", "매출액자산증가율", "순이익증가율", "영업이익률", "ROE", "ROIC", "부채비율", "이자보상배수", "차입금의존도", "매출채권회전율", "재고자산회전율", "총자본회전율", '현금영업이익', '경상활동후의현금흐름', '투자활동후의현금흐름', 'compName'
'''
# empty_df=empty_df.astype('str')

for filejson in final_corp_jsonTXT_list:
    
    # filejson = final_corp_jsonTXT_list
    filejson = json.loads(filejson)
    print(filejson['compName'])
    df = pd.json_normalize(filejson, record_path=['fnList', '2'], meta='compName')
    df = df.astype('str')
    df = df.apply(lambda x : x.str.replace(',',''), axis=1)
    # 
    temp = df['compName']
    del df['compName']
    df.insert(0, 'compName', temp)
    # 
    # temp2 = df['compName', 'CompID', 'YMD']
    # del df['compName', 'CompID', 'YMD']

    df[df.columns[3:]] = df[df.columns[3:]].apply(lambda x : pd.to_numeric(x, errors='coerce'))
    
    empty_df = pd.concat([empty_df, df], axis=0)
    # final_df = pd.concat([final_df, df], axis=0)
    # empty_df = pd.concat([empty_df, temp_df], axis=0)
    

final_df=empty_df
final_df
    # final = url_BASE + sublink
    # print(final)
    # resultdetail = session.get(final)
    # soupdetail = BeautifulSoup(resultdetail.content,'lxml')
    # print(soupdetail.select_one('title').get_text().strip())
    # table = soupdetail.select('ul.list_fs.at4')


씨이랩
두원전자통신
넛지헬스케어
야놀자
연합시스템
몬드리안에이아이
폴라리스오피스
이노룰스
KT넥스알
웅진씽크빅
한화비전
이랜드이노플
플래티어
카비
더아이엠씨
더블유컨셉코리아
엑스큐어넷
더블다운인터액티브
메이븐클라우드서비스
링네트
오누이
두플
데이터플로
어센트코리아
앤씨앤
한양대학교산학협력단
인라이플
롯데GRS
민테크
마크클라우드
충북대학교산학협력단
피처링
몰로코
에스투더블유
오케스트로
페니로이스
엑셈
대신네트웍스
다리소프트
이글루코퍼레이션
한국생산기술연구원
웨이즈원
DB하이텍
빗썸코리아
로이드케이
에이블맥스
케이앤웍스
삼정회계법인
캐롯손해보험
제니스앤컴퍼니
크리에이츠
안진회계법인
와디즈
엔파인더스
딥노이드
해줌
파수
데이타이음
고퀄
아이엔지스토리
하나금융티아이
세컨신드롬
가온플랫폼
호반프라퍼티
마루온
동녘
윈텍오토메이션
디아이티
컴퓨터메이트
뉴데이소프트
세종기술
뷰온
에이모
현대오토에버
로카 모빌리티
브레인즈컴퍼니
포시즌
에이아이더뉴트리진
텐씨엘
해양정보기술
로민
슈프리마
에스아이티테크
클라모스
플렉싱크
디노마드
아이에스동서
엔지온시스템
엠디에스인텔리전스
충북대학교병원
딥인사이트
티맵모빌리티
포뎁스
메타빌드
우주일렉트로닉스
서울대학교병원
칩스앤미디어
엠디에스테크
휴머스온
업스테이지
씨유박스
데이터메이커
데이타솔루션
인터리젠
에코마케팅
클로봇
유엔디
슈어소프트테크
스마트제어계측
브이엔티지
스터디맥스
엠씨에스텍
애자일소다
헤론헬스정보시스템
트윔
자이오넥스
씨메스
디와이엠솔루션
하이데이타
크래프트테크놀로지스
알피니언메디칼시스템
휴맥스모빌리티
머스트잇
다누시스
SK쉴더스
디알텍
미디어젠
에이아이네이션
에스에이치텍
우경정보기술
비아이매트릭스
네패스
다비오
에프원시큐리티
포트리스이노베이션
국립암센터
코리아엑스퍼트
넥슨코리아
퓨렌스
한백전자
한싹
데브시스터즈
라온피플
누리어시스템
팀스파르타
사피온코리아
에스에이티
시스콘로보틱스
뉴로메카
인터엑스
딥브레인에이아이
토페스
이노뎁
대호아이앤티
에이치비테크놀러지
삼성전자
픽소니어
아토리서치
아이유노글로벌
크래프톤
디엑스지
모빌테크

,compName,CompID,YMD,FnType,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,...,ROIC,부채비율,이자보상배수,차입금의존도,매출채권회전율,재고자산회전율,총자본회전율,현금영업이익,경상활동후의현금흐름,투자활동후의현금흐름
0,씨이랩,G26372,2020.12.31,2,8852.0,3009.0,11861.0,7582.0,1161.0,8743.0,...,7.76,280.37,5.36,19.87,2.82,NaN,1.24,3370.0,3589.0,1743.0
1,씨이랩,G26372,2021.12.31,2,26076.0,5282.0,31358.0,3816.0,3013.0,6829.0,...,-9.88,27.84,-13.24,9.46,1.72,NaN,0.27,-461.0,170.0,-4992.0
2,씨이랩,G26372,2022.12.31,2,24038.0,4796.0,28835.0,5903.0,2168.0,8071.0,...,-15.10,38.87,-33.88,7.66,4.28,NaN,0.33,-4028.0,-4459.0,-6569.0
0,두원전자통신,H07854,2020.12.31,2,9051.0,5647.0,14698.0,2397.0,2813.0,5211.0,...,8.61,54.92,25.22,21.41,46.13,35.93,1.64,1896.0,853.0,-394.0
1,두원전자통신,H07854,2021.12.31,2,11595.0,6846.0,18441.0,3487.0,1392.0,4879.0,...,24.90,35.97,76.31,2.79,41.46,28.26,2.57,3658.0,4215.0,2937.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,브이피,166186,2021.12.31,2,64969.0,17916.0,82886.0,10827.0,2518.0,13345.0,...,3.13,19.19,68.57,3.00,8.18,NaN,0.80,8001.0,7550.0,5822.0
2,브이피,166186,2022.12.31,2,58216.0,21577.0,79793.0,15891.0,4432.0,20323.0,...,5.84,34.17,56.17,5.26,8.40,51397.50,0.97,10272.0,8734.0,1456.0
0,파킹클라우드,F85404,2020.12.31,2,54388.0,69728.0,124116.0,54675.0,65235.0,119911.0,...,-31.30,2851.48,-3.22,62.10,3.30,21.49,0.50,-10015.0,-10166.0,-24353.0
1,파킹클라우드,F85404,2021.12.31,2,132931.0,69123.0,202054.0,47413.0,37634.0,85047.0,...,-8.91,72.69,-0.73,24.42,3.36,21.80,0.41,2234.0,-5883.0,-47312.0


In [79]:
print(len(final_corp_jsonTXT_list))

390


In [110]:
#final_df에서 인덱스가 2인 행들만 남기기
df2022 = final_df[final_df['YMD'] == '2022.12.31']
df2022

,compName,CompID,YMD,FnType,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,...,ROIC,부채비율,이자보상배수,차입금의존도,매출채권회전율,재고자산회전율,총자본회전율,현금영업이익,경상활동후의현금흐름,투자활동후의현금흐름
2,씨이랩,G26372,2022.12.31,2,24038.0,4796.0,28835.0,5903.0,2168.0,8071.0,...,-15.10,38.87,-33.88,7.66,4.28,NaN,0.33,-4028.0,-4459.0,-6569.0
2,두원전자통신,H07854,2022.12.31,2,9299.0,7951.0,17250.0,1987.0,1593.0,3581.0,...,0.86,26.20,5.35,3.95,30.08,10.90,1.87,-186.0,-2110.0,-2854.0
2,넛지헬스케어,HS6109,2022.12.31,2,19898.0,42281.0,62179.0,19611.0,22084.0,41696.0,...,17.81,203.56,7.75,36.10,6.01,799.63,1.11,4567.0,7488.0,3620.0
2,야놀자,C48351,2022.12.31,2,615220.0,1173058.0,1788277.0,182520.0,189620.0,372140.0,...,1.09,26.28,9.60,9.20,2.85,348.97,0.22,38476.0,103711.0,-782774.0
2,연합시스템,038618,2022.12.31,2,2063.0,9924.0,11987.0,1700.0,6466.0,8166.0,...,0.60,213.69,1.35,66.37,57.40,2.65,0.37,695.0,506.0,219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,이지스헬스케어,IN5085,2022.12.31,2,19538.0,8750.0,28288.0,9083.0,NaN,9083.0,...,-8.09,47.29,-7.93,0.00,6.35,NaN,1.14,-2293.0,-2273.0,-9364.0
2,테크빌교육,163565,2022.12.31,2,2687.0,6585.0,9272.0,4117.0,2663.0,6780.0,...,1.07,272.01,1.33,60.04,13.31,13.99,2.06,1325.0,1235.0,-1545.0
2,엔씨소프트,622850,2022.12.31,2,2521524.0,1819876.0,4341400.0,434042.0,683290.0,1117332.0,...,10.38,34.66,76.29,13.43,10.72,773.61,0.54,743400.0,493531.0,641646.0
2,브이피,166186,2022.12.31,2,58216.0,21577.0,79793.0,15891.0,4432.0,20323.0,...,5.84,34.17,56.17,5.26,8.40,51397.50,0.97,10272.0,8734.0,1456.0


In [5]:
from urllib.parse import quote
import json
def replacecomma(x):
    return x.str.replace(',','')

corp_data=pd.read_csv('saraminDF_it_companies_final.csv', encoding='utf-8')
corp_list=corp_data['IT기업2'].to_list()

final_corp_list = []
final_corp_jsonTXT_list = []
for corp in corp_list:
    # corp = corp.strip('주식회사 ')
    # corp = corp.strip(' (주) ')
    
    # print(corp)
    parsed = quote(corp)
    url_BASE = 'https://www.catch.co.kr'
    
    href = '/Comp/CompSummary/I95823'
    test_link = f'https://www.catch.co.kr/Search/SearchList?Keyword={parsed}'
    
    
    with requests.session() as session:
        result = session.get(test_link)
        soup = BeautifulSoup(result.content,'lxml')
        detaillist = soup.select_one('ul.list_corp_round.atFull')
        try:
            sublink = detaillist.select('a')[0].attrs['href']
            final_corp_list.append(detaillist.select('a')[0].text.strip())
            print(sublink)
        except Exception as e:
            print('검색결과 없음')
            continue
        urljson = 'https://www.catch.co.kr/api/v1.0/comp/compSummary/getFinancial/{}'.format(sublink.split('/')[-1])
        result = session.get(urljson)
        jsonTxt = result.text
        if jsonTxt != 'null':
            final_corp_jsonTXT_list.append(jsonTxt)
        # else:
        #     final_corp_jsonTXT_list.append(None)
        # 
        # with open('test.json', 'w', encoding='utf-8') as f:
        #     f.write(result.text)
        # 
        

/Comp/CompSummary/G26372
/Comp/CompSummary/A99AB
/Comp/CompSummary/H07854
/Comp/CompSummary/HS6109
/Comp/CompSummary/ABCD2
/Comp/CompSummary/A73E2
/Comp/CompSummary/C48351
/Comp/CompSummary/038618
/Comp/CompSummary/B0013
/Comp/CompSummary/556556
/Comp/CompSummary/497730
/Comp/CompSummary/C20187
검색결과 없음
/Comp/CompSummary/487787
/Comp/CompSummary/JB2183
/Comp/CompSummary/024518
/Comp/CompSummary/495146
/Comp/CompSummary/A88AE
/Comp/CompSummary/JB6124
/Comp/CompSummary/465936
/Comp/CompSummary/C46482
검색결과 없음
/Comp/CompSummary/123950
/Comp/CompSummary/G29102
/Comp/CompSummary/J77049
검색결과 없음
/Comp/CompSummary/133430
/Comp/CompSummary/HO3684
/Comp/CompSummary/M15136
/Comp/CompSummary/IS8879
/Comp/CompSummary/L25425
/Comp/CompSummary/023670
/Comp/CompSummary/388799
/Comp/CompSummary/I12933
/Comp/CompSummary/620475
/Comp/CompSummary/HG2578
/Comp/CompSummary/JW6339
/Comp/CompSummary/F15568
/Comp/CompSummary/JM7677
/Comp/CompSummary/M29394
검색결과 없음
검색결과 없음
/Comp/CompSummary/JH0600
/Comp/CompSumma

In [75]:
# import pickle
# 
# with open('final_corp_jsonTXT_list.pickle', 'wb') as fw:
#     pickle.dump(final_corp_jsonTXT_list, fw)
#     
# with open('final_corp_jsonTXT_list.pickle', 'rb')as fr:
#     final_corp_jsonTXT_list = pickle.load(fr)
import gc
gc.collect()

empty_df = pd.DataFrame(columns=[])

'''
계정 목록 (더 있는데 뭐가 더 있는지 찾기 어려운 상태
compID','YMD', 'FnType',"유동자산", "비유동자산", "자산총계", "유동부채", "비유동부채", "부채총계", "자본금", "자본총계", "매출액", "매출총이익", "영업이익", "영업외수익", "영업외비용", "법인세차감전이익", "법인세비용", "총당기순이익", "총자산증가율", "매출액자산증가율", "순이익증가율", "영업이익률", "ROE", "ROIC", "부채비율", "이자보상배수", "차입금의존도", "매출채권회전율", "재고자산회전율", "총자본회전율", '현금영업이익', '경상활동후의현금흐름', '투자활동후의현금흐름', 'compName'
'''
# empty_df=empty_df.astype('str')

for filejson in final_corp_jsonTXT_list:
    
    # filejson = final_corp_jsonTXT_list
    filejson = json.loads(filejson)
    print(filejson['compName'])
    df = pd.json_normalize(filejson, record_path=['fnList', '2'], meta='compName')
    df = df.astype('str')
    df = df.apply(lambda x : x.str.replace(',',''), axis=1)
    # 
    temp = df['compName']
    del df['compName']
    df.insert(0, 'compName', temp)
    # 
    # temp2 = df['compName', 'CompID', 'YMD']
    # del df['compName', 'CompID', 'YMD']

    df[df.columns[3:]] = df[df.columns[3:]].apply(lambda x : pd.to_numeric(x, errors='coerce'))
    
    empty_df = pd.concat([empty_df, df], axis=0)
    # final_df = pd.concat([final_df, df], axis=0)
    # empty_df = pd.concat([empty_df, temp_df], axis=0)
    

final_df=empty_df
final_df
    # final = url_BASE + sublink
    # print(final)
    # resultdetail = session.get(final)
    # soupdetail = BeautifulSoup(resultdetail.content,'lxml')
    # print(soupdetail.select_one('title').get_text().strip())
    # table = soupdetail.select('ul.list_fs.at4')


씨이랩
두원전자통신
넛지헬스케어
야놀자
연합시스템
몬드리안에이아이
폴라리스오피스
이노룰스
KT넥스알
웅진씽크빅
한화비전
이랜드이노플
플래티어
카비
더아이엠씨
더블유컨셉코리아
엑스큐어넷
더블다운인터액티브
메이븐클라우드서비스
링네트
오누이
두플
데이터플로
어센트코리아
앤씨앤
한양대학교산학협력단
인라이플
롯데GRS
민테크
마크클라우드
충북대학교산학협력단
피처링
몰로코
에스투더블유
오케스트로
페니로이스
엑셈
대신네트웍스
다리소프트
이글루코퍼레이션
한국생산기술연구원
웨이즈원
DB하이텍
빗썸코리아
로이드케이
에이블맥스
케이앤웍스
삼정회계법인
캐롯손해보험
제니스앤컴퍼니
크리에이츠
안진회계법인
와디즈
엔파인더스
딥노이드
해줌
파수
데이타이음
고퀄
아이엔지스토리
하나금융티아이
세컨신드롬
가온플랫폼
호반프라퍼티
마루온
동녘
윈텍오토메이션
디아이티
컴퓨터메이트
뉴데이소프트
세종기술
뷰온
에이모
현대오토에버
로카 모빌리티
브레인즈컴퍼니
포시즌
에이아이더뉴트리진
텐씨엘
해양정보기술
로민
슈프리마
에스아이티테크
클라모스
플렉싱크
디노마드
아이에스동서
엔지온시스템
엠디에스인텔리전스
충북대학교병원
딥인사이트
티맵모빌리티
포뎁스
메타빌드
우주일렉트로닉스
서울대학교병원
칩스앤미디어
엠디에스테크
휴머스온
업스테이지
씨유박스
데이터메이커
데이타솔루션
인터리젠
에코마케팅
클로봇
유엔디
슈어소프트테크
스마트제어계측
브이엔티지
스터디맥스
엠씨에스텍
애자일소다
헤론헬스정보시스템
트윔
자이오넥스
씨메스
디와이엠솔루션
하이데이타
크래프트테크놀로지스
알피니언메디칼시스템
휴맥스모빌리티
머스트잇
다누시스
SK쉴더스
디알텍
미디어젠
에이아이네이션
에스에이치텍
우경정보기술
비아이매트릭스
네패스
다비오
에프원시큐리티
포트리스이노베이션
국립암센터
코리아엑스퍼트
넥슨코리아
퓨렌스
한백전자
한싹
데브시스터즈
라온피플
누리어시스템
팀스파르타
사피온코리아
에스에이티
시스콘로보틱스
뉴로메카
인터엑스
딥브레인에이아이
토페스
이노뎁
대호아이앤티
에이치비테크놀러지
삼성전자
픽소니어
아토리서치
아이유노글로벌
크래프톤
디엑스지
모빌테크

,compName,CompID,YMD,FnType,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,...,ROIC,부채비율,이자보상배수,차입금의존도,매출채권회전율,재고자산회전율,총자본회전율,현금영업이익,경상활동후의현금흐름,투자활동후의현금흐름
0,씨이랩,G26372,2020.12.31,2,8852.0,3009.0,11861.0,7582.0,1161.0,8743.0,...,7.76,280.37,5.36,19.87,2.82,NaN,1.24,3370.0,3589.0,1743.0
1,씨이랩,G26372,2021.12.31,2,26076.0,5282.0,31358.0,3816.0,3013.0,6829.0,...,-9.88,27.84,-13.24,9.46,1.72,NaN,0.27,-461.0,170.0,-4992.0
2,씨이랩,G26372,2022.12.31,2,24038.0,4796.0,28835.0,5903.0,2168.0,8071.0,...,-15.10,38.87,-33.88,7.66,4.28,NaN,0.33,-4028.0,-4459.0,-6569.0
0,두원전자통신,H07854,2020.12.31,2,9051.0,5647.0,14698.0,2397.0,2813.0,5211.0,...,8.61,54.92,25.22,21.41,46.13,35.93,1.64,1896.0,853.0,-394.0
1,두원전자통신,H07854,2021.12.31,2,11595.0,6846.0,18441.0,3487.0,1392.0,4879.0,...,24.90,35.97,76.31,2.79,41.46,28.26,2.57,3658.0,4215.0,2937.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,브이피,166186,2021.12.31,2,64969.0,17916.0,82886.0,10827.0,2518.0,13345.0,...,3.13,19.19,68.57,3.00,8.18,NaN,0.80,8001.0,7550.0,5822.0
2,브이피,166186,2022.12.31,2,58216.0,21577.0,79793.0,15891.0,4432.0,20323.0,...,5.84,34.17,56.17,5.26,8.40,51397.50,0.97,10272.0,8734.0,1456.0
0,파킹클라우드,F85404,2020.12.31,2,54388.0,69728.0,124116.0,54675.0,65235.0,119911.0,...,-31.30,2851.48,-3.22,62.10,3.30,21.49,0.50,-10015.0,-10166.0,-24353.0
1,파킹클라우드,F85404,2021.12.31,2,132931.0,69123.0,202054.0,47413.0,37634.0,85047.0,...,-8.91,72.69,-0.73,24.42,3.36,21.80,0.41,2234.0,-5883.0,-47312.0


In [79]:
print(len(final_corp_jsonTXT_list))

390


In [110]:
#final_df에서 인덱스가 2인 행들만 남기기
df2022 = final_df[final_df['YMD'] == '2022.12.31']
df2022

,compName,CompID,YMD,FnType,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,...,ROIC,부채비율,이자보상배수,차입금의존도,매출채권회전율,재고자산회전율,총자본회전율,현금영업이익,경상활동후의현금흐름,투자활동후의현금흐름
2,씨이랩,G26372,2022.12.31,2,24038.0,4796.0,28835.0,5903.0,2168.0,8071.0,...,-15.10,38.87,-33.88,7.66,4.28,NaN,0.33,-4028.0,-4459.0,-6569.0
2,두원전자통신,H07854,2022.12.31,2,9299.0,7951.0,17250.0,1987.0,1593.0,3581.0,...,0.86,26.20,5.35,3.95,30.08,10.90,1.87,-186.0,-2110.0,-2854.0
2,넛지헬스케어,HS6109,2022.12.31,2,19898.0,42281.0,62179.0,19611.0,22084.0,41696.0,...,17.81,203.56,7.75,36.10,6.01,799.63,1.11,4567.0,7488.0,3620.0
2,야놀자,C48351,2022.12.31,2,615220.0,1173058.0,1788277.0,182520.0,189620.0,372140.0,...,1.09,26.28,9.60,9.20,2.85,348.97,0.22,38476.0,103711.0,-782774.0
2,연합시스템,038618,2022.12.31,2,2063.0,9924.0,11987.0,1700.0,6466.0,8166.0,...,0.60,213.69,1.35,66.37,57.40,2.65,0.37,695.0,506.0,219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,이지스헬스케어,IN5085,2022.12.31,2,19538.0,8750.0,28288.0,9083.0,NaN,9083.0,...,-8.09,47.29,-7.93,0.00,6.35,NaN,1.14,-2293.0,-2273.0,-9364.0
2,테크빌교육,163565,2022.12.31,2,2687.0,6585.0,9272.0,4117.0,2663.0,6780.0,...,1.07,272.01,1.33,60.04,13.31,13.99,2.06,1325.0,1235.0,-1545.0
2,엔씨소프트,622850,2022.12.31,2,2521524.0,1819876.0,4341400.0,434042.0,683290.0,1117332.0,...,10.38,34.66,76.29,13.43,10.72,773.61,0.54,743400.0,493531.0,641646.0
2,브이피,166186,2022.12.31,2,58216.0,21577.0,79793.0,15891.0,4432.0,20323.0,...,5.84,34.17,56.17,5.26,8.40,51397.50,0.97,10272.0,8734.0,1456.0


In [111]:
df2022.reset_index(drop=True, inplace=True)

In [119]:
df자산총계desc = df2022[['compName', '자산총계']].sort_values('자산총계', ascending=False)
df자산총계desc

,compName,자산총계
126,삼성전자,260083750.0
196,현대모비스,34782050.0
143,카카오,10791496.0
224,삼성SDS,7628899.0
112,넥슨코리아,6139029.0
...,...,...
240,파오스파트너스,1247.0
25,마크클라우드,1224.0
201,에이치투비즈,1198.0
19,데이터플로,1161.0


In [131]:
set(df자산낮은100['compName'].to_list()) & set(df영업이익율낮은100['compName'].to_list())


{'다비오', '메이븐클라우드서비스', '모비두', '모빌테크', '아이브스', '포티투마루'}

In [113]:
df자산총계desc[df자산총계desc['자산총계'] <= 10**7]

,compName,자산총계
224,삼성SDS,7628899.0
112,넥슨코리아,6139029.0
130,크래프톤,5774466.0
316,엔씨소프트,4341400.0
77,서울대학교병원,3729245.0
...,...,...
240,파오스파트너스,1247.0
25,마크클라우드,1224.0
201,에이치투비즈,1198.0
19,데이터플로,1161.0


In [114]:
df자산총계desc[df자산총계desc['자산총계'] <= 5*100000]

,compName,자산총계
293,네오위즈,481769.0
99,휴맥스모빌리티,470371.0
107,네패스,465878.0
218,NHN KCP,463126.0
140,테크윙,454366.0
...,...,...
240,파오스파트너스,1247.0
25,마크클라우드,1224.0
201,에이치투비즈,1198.0
19,데이터플로,1161.0


In [130]:
df자산낮은100=df자산총계desc.sort_values('자산총계', ascending=True).head(50)
df자산낮은100

,compName,자산총계
117,사피온코리아,48.0
19,데이터플로,1161.0
201,에이치투비즈,1198.0
25,마크클라우드,1224.0
240,파오스파트너스,1247.0
197,에스에스티랩,1393.0
251,아이브스,1629.0
193,딜리버드코리아,1744.0
64,로민,1875.0
42,엔파인더스,1889.0


In [116]:
df부채비율desc = df2022[['compName', '부채비율']].sort_values('부채비율', ascending=True)
df부채비율desc

,compName,부채비율
150,엑스페릭스,4.81
81,업스테이지,5.08
49,호반프라퍼티,5.57
255,포소드,5.75
96,씨메스,7.19
...,...,...
212,엑스엘게임즈,NaN
251,아이브스,NaN
276,위메프,NaN
283,럭스로보,NaN


In [129]:
df영업이익율desc = df2022[['compName', '영업이익율']].sort_values('영업이익율', ascending=True)
df영업이익율desc
df영업이익율낮은100=df영업이익율desc.head(50)
df영업이익율낮은100

,compName,영업이익율
179,포티투닷,-1700.42
99,휴맥스모빌리티,-433.41
253,루닛,-350.59
265,넥스트칩,-212.91
43,딥노이드,-193.57
122,딥브레인에이아이,-176.53
212,엑스엘게임즈,-175.16
57,에이모,-159.80
73,티맵모빌리티,-158.63
81,업스테이지,-137.96


In [119]:
df영업이익율desc = df2022[['compName', '영업이익율']].sort_values('영업이익율', ascending=False)
df영업이익율desc

,compName,영업이익율
228,네오플,79.43
35,빗썸코리아,51.07
85,에코마케팅,47.08
174,데이터유니버스,46.85
34,DB하이텍,45.88
...,...,...
265,넥스트칩,-212.91
253,루닛,-350.59
99,휴맥스모빌리티,-433.41
179,포티투닷,-1700.42


In [131]:
set(df자산낮은100['compName'].to_list()) & set(df영업이익율낮은100['compName'].to_list())


{'다비오', '메이븐클라우드서비스', '모비두', '모빌테크', '아이브스', '포티투마루'}